# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [1]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [19]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-10-25'
start_date = '2016-01-01'
# Descargar el dataframe
dis_df = yf.download('DIS',start_date, today).reset_index()

dis_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1961,2023-10-18,$85.69,$86.26,$84.60,$84.68,$84.68,10047000
1962,2023-10-19,$85.26,$85.27,$83.17,$83.25,$83.25,14316800
1963,2023-10-20,$82.90,$83.44,$82.61,$82.65,$82.65,10425200
1964,2023-10-23,$82.08,$84.03,$80.91,$83.10,$83.10,10772500
1965,2023-10-24,$83.41,$84.20,$81.74,$82.56,$82.56,11484700


In [4]:
# TAREA: Validamos que no hay datos vacíos
dis_df.isna().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [5]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = dis_df[['Date','Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds",
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [6]:
df.tail()

,ds,y
1961,2023-10-18,$85.69
1962,2023-10-19,$85.26
1963,2023-10-20,$82.90
1964,2023-10-23,$82.08
1965,2023-10-24,$83.41


In [7]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de acciones de Disney de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [8]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [9]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqnexmgum/rqm8tgji.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqnexmgum/8rgcga8b.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=57096', 'data', 'file=/tmp/tmpqnexmgum/rqm8tgji.json', 'init=/tmp/tmpqnexmgum/8rgcga8b.json', 'output', 'file=/tmp/tmpqnexmgum/prophet_modell268v6do/prophet_model-20231025103909.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:39:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:39:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [12]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future=m.make_future_dataframe(365)

In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:
future = future[future['ds'].dt.weekday < 5]  # Filtra los días de la semana (lunes a viernes)
future

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [14]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [15]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2222,2024-10-17,$58.43,$-81.07,$189.54
2223,2024-10-18,$58.45,$-80.04,$191.47
2224,2024-10-21,$58.46,$-83.56,$192.40
2225,2024-10-22,$58.47,$-84.11,$196.11
2226,2024-10-23,$58.35,$-84.70,$191.83


In [16]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

78.2331698588203

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [17]:
plot_plotly(m, forecast)

In [18]:
plot_components_plotly(m, forecast)